In [ ]:
import sqlite3
import zipfile

import numpy as np
import pandas as pd

from datetime import datetime
from tools import criaChaveAcordaoPrincipal

## Conhecer a base de dados dos acórdãos do TCU

A base de dados dos acórdãos do TCU foi disponibilizada para download público na plataforma Kaggle. Os dados dizem respeito aos acórdãos proferidos pelo TCU entre os anos de 1992 até 30/08/2019.
Acesse o endereço <https://www.kaggle.com/ferraz/acordaos-tcu> e baixe o arquivo no link Download (4GB) e salve na pasta '/dados'.
github: <https://github.com/netoferraz/acordaos-tcu>

In [ ]:
# Remover os comentário das linhas abaixo caso ainda não tenha feito a descompactação
# with zipfile.ZipFile('dados/330881_660826_bundle_archive.zip', 'r') as zip_ref:
#     zip_ref.extractall()

In [ ]:
# Cria conexão com a base de datos sqlite
conn = sqlite3.connect('dados/tcu-acordaos.db')

In [ ]:
# Cria o dataframe df_acordaos
df_acordaos = pd.read_sql_query('SELECT * from acordaos', conn)

In [ ]:
print(df_acordaos.shape, '\n')

# Mostra as colunas da base
print(df_acordaos.columns, '\n')

print(df_acordaos.info(), '\n')

In [ ]:
# Transfora em string as colunas ano_acordao
df_acordaos['ano_acordao'] = df_acordaos['ano_acordao'].map(str)

In [ ]:
df_acordaos.rename(str.upper, axis='columns', inplace=True)
df_acordaos.info()

In [ ]:
# Verifica a quantidade de registros marcados como None
print(df_acordaos[df_acordaos['ACORDAO'].values == None].shape[0])
print(df_acordaos[df_acordaos['RELATORIO'].values == None].shape[0])
print(df_acordaos[df_acordaos['VOTO'].values == None].shape[0])

# Converte os valores None para np.NaN
df_acordaos.loc[df_acordaos['ACORDAO'].values == None, 'ACORDAO'] = np.NaN
df_acordaos.loc[df_acordaos['RELATORIO'].values == None, 'RELATORIO'] = np.NaN
df_acordaos.loc[df_acordaos['VOTO'].values == None, 'VOTO'] = np.NaN

# Confirmação
print(df_acordaos[df_acordaos['ACORDAO'].values == None].shape[0])
print(df_acordaos[df_acordaos['RELATORIO'].values == None].shape[0])
print(df_acordaos[df_acordaos['VOTO'].values == None].shape[0])

In [ ]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['REPRESENTANTE_MP', 'ASSUNTO', 'UNIDADE_TECNICA', 'REPR_LEGAL', 'QUORUM', 
                                'INTERESSADO_REPONSAVEL_RECORRENTE'], axis=1)

In [ ]:
def imprime_proc_sigilo():
    print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.PROCESSO.isnull()].shape[0])
    print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.TIPO_PROCESSO == 'SIGILOSO')].shape[0])
    print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.ACORDAO == 'SIGILOSO') | (df_acordaos.ACORDAO == 'None')].shape[0])
    print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.VOTO == 'SIGILOSO')  | (df_acordaos.VOTO == 'None')].shape[0])
    print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.ENTIDADE == 'SIGILOSO')  | (df_acordaos.ENTIDADE == 'None')].shape[0])
    print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.SUMARIO == 'Número de Acórdão não utilizado'].shape[0], '\n')
    
    qtdeSigiloso = df_acordaos[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['TIPO_PROCESSO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['SUMARIO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['ACORDAO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['RELATORIO'].str.upper == 'SIGILOGO')
                               | (df_acordaos['ENTIDADE'].str.upper == 'SIGILOGO')
                               | (df_acordaos['VOTO'].str.upper == 'SIGILOGO')
                               | (df_acordaos.SUMARIO == 'Número de Acórdão não utilizado')].shape[0]

    print('Quantidade de acordãos marcados com sigilo: ', qtdeSigiloso, '\n')
    del qtdeSigiloso

In [ ]:
imprime_proc_sigilo()


print('Removendo os registros...')
df_acordaos = df_acordaos[df_acordaos.PROCESSO.notnull()]
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[df_acordaos['TIPO_PROCESSO'] =='SIGILOSO'])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['ACORDAO'] == 'SIGILOSO') | (df_acordaos['ACORDAO'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['VOTO'] == 'SIGILOSO') | (df_acordaos['VOTO'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['ENTIDADE'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['SUMARIO'] == 'Número de Acórdão não utilizado')])
print(df_acordaos.shape[0], '\n')


imprime_proc_sigilo()

In [ ]:
start_time = datetime.now()

df_acordaos['ID'] = df_acordaos.apply(criaChaveAcordaoPrincipal, axis=1)

print('Duration: {}'.format(datetime.now() - start_time))

In [ ]:
df_acordaos[df_acordaos.duplicated(['NUMERO_ACORDAO', 'PROCESSO', 'DATA_SESSAO'])]

In [ ]:
# TODO criar function para tratar quando houver mais registros duplicados
df_acordaos = df_acordaos.drop(295941)

In [ ]:
print('Quantidade de ACÓRDÃOS nulos: ', df_acordaos[df_acordaos['ACORDAO'].isnull() == True].shape[0])
print('Quantidade de registros repetidos nos campos VOTO e ACÓRDAO:', df_acordaos[df_acordaos['VOTO'].eq(df_acordaos['ACORDAO'])].shape[0])
print('Quantidade de registros repetidos nos campos RELATÓRIO e ACÓRDAO:', df_acordaos[df_acordaos['RELATORIO'].eq(df_acordaos['ACORDAO'])].shape[0])

In [ ]:
df_acordaos['DECISAO'] = df_acordaos['ACORDAO'].map(str) + ' ' + df_acordaos['VOTO'].map(str) + ' ' + df_acordaos['RELATORIO'].map(str)

In [ ]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['URN', 'ANO_ACORDAO', 'NUMERO_ACORDAO', 'RELATOR', 'PROCESSO',
                               'DATA_SESSAO', 'NUMERO_ATA', 'ENTIDADE', 'SUMARIO', 'TIPO_PROCESSO',
                               'ACORDAO', 'VOTO', 'RELATORIO'], axis=1)

df_acordaos.insert(2, 'LABEL', '')
df_acordaos.info()

In [ ]:
# Salva o arquivo em um csv compactado reduzindo o arquivo de 4GB para ~900MB
start_time = datetime.now()

df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False, compression='gzip', encoding='utf-8')

print('Duration: {}'.format(datetime.now() - start_time))

### Testes

In [ ]:
df_temp = df_acordaos.copy()

In [ ]:
print(df_temp[df_temp['DECISAO'].notnull()].shape[0])

In [ ]:
print(df_temp[df_temp['DECISAO'].isnull() == True].shape[0])

In [ ]:
df_dupl = df_temp[df_temp.duplicated(['DECISAO'])]
df_dupl.info()

In [ ]:
df_dupl

In [ ]:
# Lê o arquivo de um csv compactado
df_acordaos = pd.read_csv('dados/df_acordaos.csv', sep='|', compression='gzip')